<a href="https://colab.research.google.com/github/AkashCreator/mydata/blob/main/pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=a15fa8231d153bedc96c4a8ca1785f0d110ab077cac3f0f9d51ce1ecfd38face
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('myapp1').getOrCreate()

In [4]:
spark

In [5]:
df=spark.read.csv('./sample_data/titanic_train.csv',header=True,inferSchema=True)

In [6]:
df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [7]:
df.dtypes

[('PassengerId', 'int'),
 ('Survived', 'int'),
 ('Pclass', 'int'),
 ('Name', 'string'),
 ('Sex', 'string'),
 ('Age', 'double'),
 ('SibSp', 'int'),
 ('Parch', 'int'),
 ('Ticket', 'string'),
 ('Fare', 'double'),
 ('Cabin', 'string'),
 ('Embarked', 'string')]

In [8]:
df.createOrReplaceTempView("my_titanic")

In [9]:
data1=spark.sql('SELECT * from my_titanic')

In [10]:
data1.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [11]:

data2=spark.sql("""SELECT COUNT(*) AS count,Sex as Gender
                         from my_titanic
                             GROUP BY Sex, """)

In [12]:
data2.show()

+-----+------+
|count|Gender|
+-----+------+
|  314|female|
|  577|  male|
+-----+------+



In [20]:
# Gender wise survival data where cabin is assigned
data3=spark.sql("""SELECT COUNT(*) As ReservedCount,Sex,Survived FROM my_titanic
                   WHERE Cabin is NOT NULL GROUP BY Sex,Survived Order BY Survived,Sex desc""" )
data3.show()

+-------------+------+--------+
|ReservedCount|   Sex|Survived|
+-------------+------+--------+
|           62|  male|       0|
|            6|female|       0|
|           45|  male|       1|
|           91|female|       1|
+-------------+------+--------+



In [41]:
from pyspark.sql.functions import *
df7=df.filter(col("Cabin").isNull())
df7.count()


687

In [55]:
#Total passengers having cabin booked
customershavingdata=df.count()-df7.count()
print(customershavingdata);

204
